## Vinculando o Knowledge Graph com o LLM para permitir busca semântica

Depois de inserir os dados no banco, criar seus embeddings e seu vetor de índices, agora podemos fazer experimentos para permitir a implementação da busca semântica.

In [1]:
from neo4j import GraphDatabase
import os
from dotenv import load_dotenv

In [2]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint, HuggingFaceEmbeddings
from langchain_neo4j import GraphCypherQAChain, Neo4jGraph, Neo4jVector
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

In [3]:
from huggingface_hub import login
login()

In [4]:
load_dotenv()
model_name = os.getenv("MODEL_NAME")
embedding_provider = HuggingFaceEmbeddings(
    model_name=model_name
)

In [5]:
graph = Neo4jGraph(
    url=os.getenv('NEO4J_CONNECTION'),
    username=os.getenv('NEO4J_USERNAME'),
    password=os.getenv('NEO4J_PASSWORD')
)

### Testando a busca de artigos por similarity search

In [6]:
prod_titulo_vector = Neo4jVector.from_existing_index(
    embedding_provider,
    graph=graph,
    index_name = 'producoes',
    embedding_node_property='embedding',
    text_node_property='titulo',
)

result = prod_titulo_vector.similarity_search("artificial jogos")

for doc in result:
    print(doc.metadata["autor"], "-", doc.page_content)

Guilherme de Almeida Xavier - Jogos Eletrônicos na Teoria dos Eixos
Bruno Feijo - Introdução à Ciência da Computação com Jogos
Bruno Feijo - Visualização, Simulação e Games
Guilherme de Almeida Xavier - O simbólico interativo no jogo eletrônico


In [7]:
retriever = prod_titulo_vector.as_retriever(search_type="similarity", search_kwargs={"k": 3})

Vemos que por similarity search já conseguimos alguns resultados bem interessantes de artigos envolvendo os assuntos, mas acho que precisariamos de mais um refinamento para funcionar melhor.

## Testando busca semântica com a similarity_search

In [8]:
import spacy

# Carregue o modelo em português
nlp = spacy.load("pt_core_news_sm")

def busca_artigos(query: str):
    doc = nlp(query.lower())
    filtered_tokens = [token.text for token in doc if not token.is_stop and token.is_alpha]
    important_tokens = ' '.join([str(x) for x in filtered_tokens])

    result = prod_titulo_vector.similarity_search(important_tokens)

    for doc in result:
        print(doc.metadata["autor"], "-", doc.page_content)

busca_artigos("Artigos sobre linguagens de programação")

Marcos Vianna Villas - Programação: conceitos, técnicas e linguagens
Roberto Ierusalimschy - A Implementação da Linguagem de Programação SLOTH
Edward Hermann Haeusler - Paradigmas de linguagens de programação: uma abordagem geométrica
Renato Fontoura de Gusmão Cerqueira - Utilizando Anotações em Linguagens Orientadas a Objetos para Suporte à Programação Orientada a Componentes


### Testando importação com retrieval chains

In [9]:
llm = HuggingFaceEndpoint(
    repo_id="bigscience/bloom",
    task="text-generation",
)

chat = ChatHuggingFace(llm=llm, verbose=True)

In [10]:
titulo_retriever = RetrievalQA.from_llm(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

response = titulo_retriever.invoke(
    {"query": "Qual o artigo mais recente sobre inteligência artificial?"}
)

print(response)

C:\dev\IC-BioBD\ic-env\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


{'query': 'Qual o artigo mais recente sobre inteligência artificial?', 'result': ' O artigo mais recente sobre inteligência artificial no contexto fornecido é "A Inovação com Dados: a Experiência do Usuário com Sistemas Baseados em Inteligência Artificial".', 'source_documents': [Document(metadata={'lattes_criador': 5462014719461367, 'autor': 'Maria Manuela Rupp Quaresma', 'tipo': 'Article', 'ano': 2021}, page_content='A Inovação com Dados: a Experiência do Usuário com Sistemas Baseados em Inteligência Artificial'), Document(metadata={'lattes_criador': 1728746187630338, 'autor': 'Bernardo Pereira Nunes', 'tipo': 'Article', 'ano': 2023}, page_content='AAIN Generative Artificial Intelligence Guidelines'), Document(metadata={'lattes_criador': 5462014719461367, 'autor': 'Maria Manuela Rupp Quaresma', 'tipo': 'Article', 'ano': 2022}, page_content='Interação com redes sociais, bem-estar e a terceira onda de evolução da Inteligência Artificial')]}


In [23]:
CYPHER_GENERATION_PROMPT = """Tarefa: Gere uma query em Cypher para consultar um banco de dados de grafos. Não retorne nada além da query Cypher.
                Schema: {schema}
                Question: {question}

                Instruções:
                * Assegure-se de que a query Cypher seja sintaticamente correta e use somente comandos válidos em Cypher.
                * Seja o mais conciso possível nas respostas, retorne somente a resposta da pergunta.
                * Retorne sempre a query em Cypher.
                * Utilize apenas operadores válidos.
                ---
                Cypher Query:"""


cypher_generation_prompt = PromptTemplate(template=CYPHER_GENERATION_PROMPT, input_variables=["schema", "question"])

In [24]:
cypher_chain = GraphCypherQAChain.from_llm(
    llm,
    graph=graph,
    cypher_prompt=cypher_generation_prompt,
    verbose=False,
    allow_dangerous_requests=True
)

result = cypher_chain.invoke({"query": "Artigos sobre inteligencia artificiial"})
print(result)

C:\dev\IC-BioBD\ic-env\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


{'query': 'Artigos sobre inteligencia artificiial', 'result': ''}


C:\dev\IC-BioBD\ic-env\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)
